# Índice de Contingência


## Introdução

Este relatório descreve o desenvolvimento do Índice de Contingência (IC) para um grande projeto de engenharia, utilizando metodologias avançadas como Simulação de Monte Carlo e Lógica Fuzzy. O objetivo é fornecer uma análise competitiva do IC, diante dos riscos envolvidos, assegurando a competitividade da concorrência e sustentabilidade financeira do projeto.

## Sumário
1. Metodologia
2. Estabelecendo ICs
3. Simulação de Cenários com ICs comparativos
4. Folha de Resultados



## 1. Metodologia

Foi considerada uma mescla de 3 abordagens para tornar o índice de contingência mais apurado para situações em que não se tem histórico de dados:
- Simulação de Monte Carlo 
- Lógica Fuzzy
- Análise de Sensibilidade de ICC

### 1.1. Dados

Os dados foram retirados da matriz de risco realizada pela empresa. Eles estão itemzidados de 1 a 65. As classificações dos riscos serão em:
- Impacto no Custo (IF)
- Impacto no cronograma (AC)
- Impacto combinado de custo e cronograma (IFAC)
- Probabilidade de ocorrência (MC)

#### 1.1.1. Faixas de Valores para as Variáveis

- Impacto Financeiro (em % do total do projeto = 1.100 bilhão ):
    - Baixo: (≤1 milhão de reais)
    - Médio: (>1 a ≤5 milhões de reais)
    - Alto: (>5 milhões de reais)
- Atraso no Cronograma (em semanas):
    - Baixo: ≤4 semanas
    - Médio: >4 a ≤8 semanas
    - Alto: >8 semanas

### 1.3. Encontrando a probabilidade de ocorrência (MC)

A média e DP estão configurados para refletir uma probabilidade razoável de ocorrência, o que é crucial para planejamento de contingências e alocação de recursos. Os limiares estão ajustados para diferenciar efetivamente entre os impactos esperados de riscos baixos e médios.

#### 1. Riscos Baixos:

Probabilidade Média: Aproximadamente 15%

A probabilidade de ocorrência de riscos baixos variando em torno de 15% sugere uma frequência moderada de eventos menores, que são esperados e gerenciáveis dentro do contexto da obra. Essa frequência é adequada dado o limiar e os parâmetros definidos, indicando que pequenos contratempos são prováveis mas não frequentemente severos.

#### 2. Riscos Médios:

Probabilidade Média: Aproximadamente 11.5%

Riscos médios apresentando uma probabilidade de cerca de 11.5% de ocorrência são realistas para um projeto desta envergadura. Isto reflete uma boa calibração no balanceamento entre a severidade e a frequência esperada desses eventos, que devem ser considerados no planejamento e preparação da gestão de riscos.

#### 3. Riscos Altos:

Probabilidade Média: Aproximadamente 1%

A probabilidade muito baixa para riscos altos é desejável e reflete adequadamente a natureza desses riscos; altamente impactantes mas raros. A configuração dos parâmetros foi eficaz para garantir que tais riscos só se manifestem sob condições extremas, alinhado com o alto limiar estabelecido.
Sendo assim, para encontrar esta probabilidade, optou-se por rodar 30.000 simulações sob a metodologia de monte carlo com o acréscimo dos parâmetros abaixo: 

##### Riscos Baixos:
•	Média: 0.015 - probabilidade de ocorrência.
•	Desvio padrão: 0.005 - diminuir a variabilidade.
•	Limiar: 0.015 - confirma a ocorrência do risco baixo sem ser excessivamente fácil de superar.


##### Riscos Médios:
•	Média: 0.00115 - refletir a chance de ocorrência.
•	Desvio padrão: 0.01 - adicionar mais variabilidade.
•	Limiar: 0.02 - maior chance de superação, sem ser demasiado baixo.


##### Riscos Altos:
•	Média: 0.005 - probabilidade de ocorrência.
•	Desvio padrão: 0.015 - Aumentado significativamente para refletir a alta variabilidade e severidade destes riscos quando ocorrem.
•	Limiar: 0.04 - permitir que eventos realmente graves sejam capturados pela simulação mais frequentemente.


### 1.4. Nova planilha compilada

Então, tem-se agora a planilha riscos, com o MC incorporado a cada risco e o impacto combinado AC/IF/ACIF. Ela se chama resultado_simulacao



## 2. Estabelecendo ICs - MONTE CARLO 

### 2.1. IC – Monte Carlo
import numpy as np
import pandas as pd

# Configuração para mostrar todas as linhas
pd.set_option('display.max_rows', None)

# Ajustes nas configurações de simulação baseadas no impacto
impact_settings = {
    'baixo': {'mean': 0.01, 'std': 0.005, 'threshold': 0.015},
    'médio': {'mean': 0.008, 'std': 0.01, 'threshold': 0.02},
    'alto': {'mean': 0.005, 'std': 0.015, 'threshold': 0.04}
}

# Função para simular a probabilidade de ocorrência de um risco
def simulate_risk(risk_impact, n=30000):
    settings = impact_settings[risk_impact.strip().lower()]
    simulations = np.random.normal(loc=settings['mean'], scale=settings['std'], size=n)
    probability = np.mean(simulations > settings['threshold'])
    return probability

# Leitura e aplicação da simulação
file_path = resultado.simulação.csv'
df = pd.read_csv(file_path, sep=';')
df.columns = ['CÓDIGO RISCO', 'IMPACTO']
df['IMPACTO'] = df['IMPACTO'].str.strip().str.lower()
df['Simulated Prob'] = df['IMPACTO'].apply(simulate_risk)

print("Dados carregados com sucesso:")
print(df)

# Definir pesos para diferentes níveis de impacto
weights = {'baixo': 0.5, 'médio': 1.0, 'alto': 1.5}

# Aplicar pesos às probabilidades simuladas conforme o impacto
df['Weighted Prob'] = df['Simulated Prob'].multiply(df['IMPACTO'].map(weights))

# Calcular o Índice de Contingência como a soma das probabilidades ponderadas
IC = df['Weighted Prob'].sum()

print(f"Índice de Contingência Calculado: {IC:.2f}%")

try:
    df.to_excel(caminho, index=False)
    print("Resultados salvos com sucesso em 'resultado_simulacao.xlsx'.")
except Exception as e:
    print("Falha ao salvar o arquivo Excel:", e)

## 2. Estabelecendo ICs - LÓGICA FUZZY

### 2.2. IC – Lógica Fuzzy

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Dados embutidos diretamente no código
dados = [
] # base de dados de riscos 

# Mapeamento dos impactos e probabilidade
impact_map = {'baixo': 0.25, 'médio': 0.50, 'alto': 0.75}
impacts = [impact_map[item["Impacto"]] for item in dados]
probabilities = [item["Probabilidade"] for item in dados]

# Antecedentes e Consequente
impacto = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'impacto')
probabilidade = ctrl.Antecedent(np.arange(0, 0.21, 0.01), 'probabilidade')
IC_fuzzy = ctrl.Consequent(np.arange(0, 10.1, 0.1), 'IC_fuzzy')  # Saída de 0% a 10%

# Funções de pertinência
impacto.automf(3, names=['baixo', 'médio', 'alto'])
probabilidade.automf(3, names=['baixa', 'média', 'alta'])

# Customização das funções de pertinência para IC_fuzzy
IC_fuzzy['baixo'] = fuzz.trimf(IC_fuzzy.universe, [0, 2, 4])
IC_fuzzy['médio'] = fuzz.trimf(IC_fuzzy.universe, [2, 4, 6])
IC_fuzzy['alto'] = fuzz.trimf(IC_fuzzy.universe, [4, 6, 8])

# Regras
rule1 = ctrl.Rule(impacto['baixo'] & probabilidade['baixa'], IC_fuzzy['baixo'])
rule2 = ctrl.Rule(impacto['baixo'] & probabilidade['média'], IC_fuzzy['baixo'])
rule3 = ctrl.Rule(impacto['baixo'] & probabilidade['alta'], IC_fuzzy['médio'])
rule4 = ctrl.Rule(impacto['médio'] & probabilidade['baixa'], IC_fuzzy['baixo'])
rule5 = ctrl.Rule(impacto['médio'] & probabilidade['média'], IC_fuzzy['médio'])
rule6 = ctrl.Rule(impacto['médio'] & probabilidade['alta'], IC_fuzzy['alto'])
rule7 = ctrl.Rule(impacto['alto'] & probabilidade['baixa'], IC_fuzzy['médio'])
rule8 = ctrl.Rule(impacto['alto'] & probabilidade['média'], IC_fuzzy['alto'])
rule9 = ctrl.Rule(impacto['alto'] & probabilidade['alta'], IC_fuzzy['alto'])

system = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
simulator = ctrl.ControlSystemSimulation(system)

# Aplicação das regras e cálculo do IC
results = []
for imp, prob in zip(impacts, probabilities):
    simulator.input['impacto'] = imp
    simulator.input['probabilidade'] = prob
    simulator.compute()
    results.append(simulator.output['IC_fuzzy'])

# Calculando o IC agregado
IC_agregado = np.mean(results)
print(f"Índice de Contingência Agregado: {IC_agregado:.2f}%")



## 3 Simulação de Cenários com ICs comparativos

Para acumular mais indicadores que pudessem estabelecer a performance dos índices de contingência escolhidos, foram criados: 

### 3.1	Medição da competitividade da empresa

Feita uma nova simulação de Monte Carlo, agora com 10.000 replicações, considerando as premissas: 

•	O custo direto da obra é de 1.100 bilhão de reais;
•	Tenho outros 3 concorrentes (Concorrente 1, concorrente 2, concorrente 3) que podem variar deste valor -10% a +10%; 
•	Eles podem considerar índices de contingência de IC 1% a 10% sobre o valor de 1.100 bilhão. Ou seja, o valor final da obra é 1 bilhão + IC aplicado. 

Os indicadores abaixo avaliarão se a empresa tem capacidade financeira para lidar com situações onde o IC pode ser insuficiente 

### 3.2 Análise de Reserva de Contingência

Cálculo da Reserva de Contingência Necessária:

- Reserva por Risco: `Reserva_{risco} = \text{Probabilidade} \times \text{Impacto Financeiro}`
- Reserva Total de Contingência: `Reserva Total = \sum (\text{Reserva}_{risco})`

 
Armazenado o valor de reserva total, é possível obter o Índice de Cobertura de Contingência (ICC)
Cálculo do ICC:

- ICC = `\frac{\text{IC Estabelecido}}{\text{Reserva Total Necessária}}`
 
Sendo, 
•	ICC > 1: Indica que o IC é suficiente para cobrir as contingências estimadas
•	ICC < 1: Indica insuficiência, sugerindo risco de sustentabilidade financeira

### 3.3 Simulação de Monte Carlo para Estimar a Adequação do IC

Foi realizada a simulação das demandas de contingência, considerando cenários onde múltiplos riscos ocorrem com base nas suas probabilidades e então calcula-se o custo total de contingência para cada cenário.

### 3.4 Probabilidade de Insuficiência do IC:

É determinada então a proporção de simulações onde o custo total de contingências excede o IC estabelecido pelos métodos Fuzzy e Monte Carlo. 

Ou seja, os cenários os quais os riscos combinados ocorrem de modo a ultrapassar o custo de contingência anteriormente previsto. 

### 3.5 Análise de Sensibilidade

#### a. Variação do IC:

Analise como mudanças no IC afetam o ICC e a probabilidade de insuficiência (usando Monte Carlo).

#### b. Threshold (limiar) de Segurança:

Estabeleça um limite mínimo para o ICC (e.g., ICC > 1.1) que ofereça uma margem de segurança.



## 4. Resultados das simulações de competitividade

### 3.6 Código

import numpy as np
import pandas as pd

# Parâmetros iniciais
custo_base = 1.1e9  # 1.100 bilhão de reais
ic_pessoal = sorted([0.03, 0.0415, 0.0549])  # ICs ordenados do menor para o maior
ic_min = 0.01 * 1.1e9  # Mínimo de 1% de 1.100 bilhão
ic_max = 0.08 * 1.1e9  # Máximo de 8% de 1.100 bilhão
ic_step = 0.01 * 1.1e9  # Incremento
threshold_seguranca = 1.1  # Threshold de segurança para o ICC
percentual_resultado_liquido = 0.20  # 20% do resultado líquido considerado
resultado_liquido = custo_base * percentual_resultado_liquido

# Definição dos dados
dados = [
 
] # tabela de riscos 

# Conversão para DataFrame
df = pd.DataFrame(dados)
df['Impacto Financeiro'] = df['Impacto'].map({'baixo': 1e6, 'médio': 3e6, 'alto': 5.5e6})

# Simulação de Monte Carlo
n_simulations = 10000
simulated_costs = np.zeros(n_simulations)

# Realizar simulações
for i in range(n_simulations):
    occurrences = np.random.rand(len(df)) < df['Probabilidade']
    simulated_costs[i] = (occurrences * df['Impacto Financeiro']).sum()


# Análise para cada IC estabelecido
for ic in [1.1e9 * x for x in [0.03, 0.0415, 0.0549]]:
    ic_coverage = simulated_costs <= ic
    coverage_rate = ic_coverage.mean() * 100
    excess_cost = simulated_costs[~ic_coverage]
    average_excess_cost = excess_cost.mean() if np.any(~ic_coverage) else 0
    icc_value = df['Probabilidade'].dot(df['Impacto Financeiro']) / ic
    insufficiency_probability = 100 * (1 - coverage_rate / 100)
    ic_percentage = (ic / custo_base) * 100  # Convertendo IC para percentual
    exposure_percentage = (average_excess_cost / resultado_liquido) * 100 if average_excess_cost > 0 else 0

    print(f"Para um IC de {ic_percentage:.2f}%:")
    print(f" - ICC (Índice de Cobertura Contingencial): {icc_value:.2f} (Limiar de segurança: {threshold_seguranca})")
    print(f" - Cobertura de riscos: {coverage_rate:.2f}%")
    print(f" - Probabilidade de insuficiência: {insufficiency_probability:.2f}%")
    print(f" - Exposição do resultado líquido à insuficiência: {exposure_percentage:.2f}% do resultado líquido")

# Análise de Sensibilidade
ic_min = 0.01 * custo_base
ic_max = 0.08 * custo_base
ics_variados = np.linspace(ic_min, ic_max, 10)  # Ajuste para obter mais valores

sensitivity_results = {}
excess_costs = {}

for ic in ics_variados:
    ic_coverage = simulated_costs <= ic
    excess_cost = simulated_costs[simulated_costs > ic] - ic if np.any(simulated_costs > ic) else np.array([0])
    average_excess_cost = np.mean(excess_cost)
    icc_varied = df['Probabilidade'].dot(df['Impacto Financeiro']) / ic
    sensitivity_results[ic] = icc_varied
    excess_costs[ic] = average_excess_cost

# Ordenando os resultados para apresentação
sorted_ics = sorted(sensitivity_results.items(), key=lambda x: x[1])  # Ordenando pelo ICC

print("\nSensitivity Analysis:")
for ic, icc in sorted_ics[:10]:  # Mostrando apenas os dez primeiros resultados
    ic_percentage = (ic / custo_base) * 100
    exposure_percentage = (excess_costs[ic] / resultado_liquido) * 100
    print(f" - Para um IC de {ic_percentage:.2f}%, o ICC ajustado é {icc:.2f}")
    print(f" - Exposição do resultado líquido à insuficiência: {exposure_percentage:.2f}% do resultado líquido")



## 4. FOLHA DE RESULTADOS

Folha de resultados:
O IC com maior competitividade é de 3.33%. Foi avaliada a performance de outros três valores para o índice de contingência: 3%, 4.15% e 5.49%.

- IC (Índice de Contingência) – Índice percentual de contingência a ser aplicado sobre o valor global do contrato
- ICC (Índice de Cobertura Contingencial) – Índice que mede a distância entre a contingência aplicada e o valor total necessário caso todos os riscos ocorram simultaneamente.

### Análise de Competitividade

- Quando 3.00% de IC, as chances de ganho são 6.18% maior em relação a IC 4.15% e 10.92% maior em relação a IC 5.49%.
- Quando 4.15% de IC, as chances de ganho são 6.18% menor em relação a IC 3.00% e 4.74% maior em relação a IC 5.49%.
- Quando 5.49% de IC, as chances de ganho são 10.92% menor em relação a IC 3.00% e 4.74% menor em relação a IC 4.15%.

### Performance e exposição do resultado líquido

- Para um IC de 3.00%:
    - ICC (Índice de Cobertura Contingencial): 0.42 (Limiar de segurança: 1.1)
    - Cobertura de riscos: 99.88%
    - Probabilidade de insuficiência: 0.12%
    - Exposição do resultado líquido à insuficiência: 15.98% do resultado líquido

- Para um IC de 4.15% e 5.49%:
    - ICC (Índice de Cobertura Contingencial): 0.31 (Limiar de segurança: 1.1)
    - ICC (Índice de Cobertura Contingencial): 0.23 (Limiar de segurança: 1.1)
    - Cobertura de riscos: 100.00%
    - Probabilidade de insuficiência: 0.00%
    - Exposição do resultado líquido à insuficiência: 0.00% do resultado líquido


### Análise de Sensibilidade:

- Para um IC de 8.00% o ICC ajustado é 0.16 e  compromete 0.00% do resultado líquido
- Para um IC de 4.89% o ICC ajustado é 0.26 e  compromete 0.00% do resultado líquido
- Para um IC de 3.33% o ICC ajustado é 0.38 e compromete 0.00% do resultado líquido
- Para um IC de 2.56% o ICC ajustado é 0.50 e compromete 1.20% do resultado líquido
- Para um IC de 1.78% o ICC ajustado é 0.71 e compromete 1.49% do resultado líquido
- Para um IC de 1.00% o ICC ajustado é 1.27 e compromete 2.68% do resultado líquido


## CONCLUSÃO

Além de estabelecer o IC pelos métodos já conhecidos, optou-se por explorar a exposição destes índices diante do aspecto financeiro do projeto e comercial, quanto as chances de ganhar ou perder aquela concorrência. 

Mesmo os métodos tendo indicado ICs maiores de 3.30%, para este cenário e condições expostas acima, 3.30% é o menor valor cuja exposição da margem é praticamente nula e ainda prioriza as chances de ganho de concorrência. 